In [14]:
import sys
sys.path.append('../main')
import sql
import pandas as pd
from functools import reduce

In [15]:
q='''select * from cve c
    join memory_cwe mc on c.CWE = mc.CWE
    join project p on c.project_id = p.id
    where memory = 1;'''
df=pd.DataFrame(sql.execute(q))
cvss3 = df.groupby('name')[['cvss3_score']].median()
cvss2 = df.groupby('name')[['cvss2_score']].median()
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[cvss2, cvss3])
df = df.rename(columns={'cvss3_score':'memory_cvss3','cvss2_score':'memory_cvss2'})
memory = df
memory

,memory_cvss2,memory_cvss3
name,,
Firefox,7.50,9.8
LibreOffice,7.50,9.8
Linux,7.20,7.8
OpenCV,6.80,8.8
Samba,5.00,6.5
Thunderbird,7.55,9.8


In [16]:
q='''select name, count(*) as nonmemory
from cve c
join project p on c.project_id = p.id
join memory_cwe mc on c.CWE = mc.CWE
where memory is null
group by name;'''
nmc=pd.DataFrame(sql.execute(q))

In [17]:
q='''select * from cve c
    join memory_cwe mc on c.CWE = mc.CWE
    join project p on c.project_id = p.id
    where memory is null;'''
df=pd.DataFrame(sql.execute(q))
cvss3 = df.groupby('name')[['cvss3_score']].median()
cvss2 = df.groupby('name')[['cvss2_score']].median()
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[nmc, cvss2, cvss3])
df = df.rename(columns={'cvss3_score':'nonmemory_cvss3','cvss2_score':'nonmemory_cvss2'})
nonmemory = df
nonmemory

,name,nonmemory,nonmemory_cvss2,nonmemory_cvss3
0,Firefox,737,5.00,6.1
1,Thunderbird,308,5.00,6.5
2,Linux,1150,4.90,6.7
3,Samba,84,5.00,6.5
4,LibreOffice,20,6.80,7.8
5,OpenCV,6,4.65,6.7
6,Kodi,3,4.30,6.1


In [18]:
q='''select name, count(*) as memory
from cve c
join project p on c.project_id = p.id
join memory_cwe mc on c.CWE = mc.CWE
where memory=1
group by name;'''
mc=pd.DataFrame(sql.execute(q))


In [19]:
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[mc,memory, nonmemory])
df

,name,memory,memory_cvss2,memory_cvss3,nonmemory,nonmemory_cvss2,nonmemory_cvss3
0,Firefox,624,7.50,9.8,737,5.00,6.1
1,Thunderbird,426,7.55,9.8,308,5.00,6.5
2,Linux,1427,7.20,7.8,1150,4.90,6.7
3,Samba,41,5.00,6.5,84,5.00,6.5
4,LibreOffice,16,7.50,9.8,20,6.80,7.8
5,OpenCV,25,6.80,8.8,6,4.65,6.7


In [20]:
from scipy.stats import mannwhitneyu
stat, p = mannwhitneyu(df.memory_cvss2,df.nonmemory_cvss2)
stat, p, df.memory_cvss2.median(), df.nonmemory_cvss2.median()

(3.0, 0.00884228658785868, 7.35, 5.0)

In [21]:
stat, p = mannwhitneyu(df.memory_cvss3,df.nonmemory_cvss3)
stat, p, df.memory_cvss3.median(), df.nonmemory_cvss3.median()

(4.5, 0.017043467129899254, 9.3, 6.6)

In [22]:
from tabulate import tabulate
df=df.sort_values('memory', ascending=False)
#df=df.rename(columns={'name':'Project', 'memory':'Memory CVEs','memory_cvss2':'Median CVSS2 score','memory_cvss3':'Median CVSS3 score'})
#df=df.drop(columns=['nonmemory_cvss2','nonmemory_cvss3'])
print(tabulate(df, tablefmt='latex',headers='keys',showindex=False))

\begin{tabular}{lrrrrrr}
\hline
 name        &   memory &   memory\_cvss2 &   memory\_cvss3 &   nonmemory &   nonmemory\_cvss2 &   nonmemory\_cvss3 \\
\hline
 Linux       &     1427 &           7.2  &            7.8 &        1150 &              4.9  &               6.7 \\
 Firefox     &      624 &           7.5  &            9.8 &         737 &              5    &               6.1 \\
 Thunderbird &      426 &           7.55 &            9.8 &         308 &              5    &               6.5 \\
 Samba       &       41 &           5    &            6.5 &          84 &              5    &               6.5 \\
 OpenCV      &       25 &           6.8  &            8.8 &           6 &              4.65 &               6.7 \\
 LibreOffice &       16 &           7.5  &            9.8 &          20 &              6.8  &               7.8 \\
\hline
\end{tabular}
